In [1]:
import pandas as pd
import plotly.graph_objs as go

In [56]:
df_circuits = pd.read_csv('./data/circuits.csv', header = 0)
df_drivers = pd.read_csv('./data/drivers.csv',header = 0)
df_races = pd.read_csv('./data/races.csv',header = 0, delimiter = ';')
df_results = pd.read_csv('./data/results.csv',header = 0)
#df_circuits = pd.read_csv('./data/circuits.csv')
df_laps = pd.read_csv('./data/lap_times.csv',header = 0)

df_races.set_index('raceId', inplace=True)
df_results.set_index('resultId', inplace=True)
df_drivers.set_index('driverId', inplace=True)

In [3]:
df_final = pd.read_excel('./data/circuits_drivers_races_constructors_results.xlsx')

In [5]:
circuit_name = 'Bahrain International Circuit'

### Circuit Information

In [172]:
circuit_name = 'Silverstone Circuit'

In [173]:
df_final_circuit =  df_final.loc[(df_final['circuits.name'] == str(circuit_name))]


In [174]:
df_final_circuit.columns

Index(['resultId', 'raceId', 'driverId', 'positionText', 'positionOrder',
       'races.raceId', 'races.year', 'races.circuitId', 'races.name',
       'drivers.forename', 'drivers.surname', 'drivers.nationality',
       'circuits.name', 'circuits.location', 'circuits.country',
       'circuits.lat', 'circuits.lng', 'constructors.constructorRef',
       'constructors.name', 'constructors.nationality', 'constructorId'],
      dtype='object')

In [175]:
df_final_circuit['drivers.fullname'] = df_final_circuit[['drivers.forename','drivers.surname']].apply(lambda x: ' '.join(x), axis=1)

C:\Users\Admin\AppData\Local\Temp/ipykernel_9412/3853478060.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [176]:
city = df_final_circuit['circuits.location'].unique()[0]
country = df_final_circuit['circuits.country'].unique()[0]

In [177]:
best_driver = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby('drivers.fullname')['drivers.fullname'].count().sort_values(ascending = False).index.tolist()[0]

In [178]:
best_constructor = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).index.tolist()[0]

In [179]:
number_ret = len(df_final_circuit[df_final_circuit['positionText'] == 'R'])

In [197]:
# erro possivel aqui --- averiguar
recent_driver_win, year_win = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby(['drivers.fullname','races.year'])['drivers.fullname'].count().sort_values(ascending = False).index.tolist()[0]

In [187]:
recent_driver_win, year_win

('Alain Prost', 1983)

### No race result vs finished race

In [188]:
circuit_name = 'Silverstone Circuit'

In [189]:
ret_list = ['R', 'F', 'W', 'N', 'D','E']

In [190]:
# plot de dnfs por equipa
plot_df_ret = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'].isin(ret_list))].groupby('constructors.name').count()
plot_df_ret.reset_index(inplace=True)
bad_list = plot_df_ret['constructors.name'].unique()

In [191]:
# plot de corridas acabadas por equipa 
plot_df_fin = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['constructors.name'].isin(bad_list))].groupby('constructors.name').count()
plot_df_fin.reset_index(inplace=True)

In [192]:
# NÃO ESQUECER QUE PODE HAVER CIRCUITOS SEM DNFS

In [193]:
plot_percentages = pd.DataFrame(columns=['constructor.names','dnf','nraces'])

In [194]:
plot_df_ret.columns

Index(['constructors.name', 'resultId', 'raceId', 'driverId', 'positionText',
       'positionOrder', 'races.raceId', 'races.year', 'races.circuitId',
       'races.name', 'drivers.forename', 'drivers.surname',
       'drivers.nationality', 'circuits.name', 'circuits.location',
       'circuits.country', 'circuits.lat', 'circuits.lng',
       'constructors.constructorRef', 'constructors.nationality',
       'constructorId', 'drivers.fullname'],
      dtype='object')

In [195]:
plot_percentages['constructor.names'] = plot_df_ret['constructors.name']
plot_percentages['dnf'] = plot_df_ret['resultId'] / plot_df_fin['resultId']
plot_percentages['nraces'] = 1-plot_percentages['dnf']

In [196]:
fig = go.Figure(data=[
    go.Bar(name='DNF', x=plot_percentages['constructor.names'], y=plot_percentages['dnf'] ),
    go.Bar(name='Nº OF RACES', x=plot_percentages['constructor.names'], y=plot_percentages['nraces'])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

### Lap times

In [77]:
races_list = df_final_circuit['raceId'].unique()

In [61]:
df_laps_circuit = df_laps.loc[(df_laps['raceId'].isin(races_list))]

In [67]:
df_laps_circuit[['minute', 'seconds']] = df_laps_circuit['time'].str.split(':', expand=True)

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [76]:
df_final_circuit[df_final_circuit['raceId'] == 863]

,resultId,raceId,driverId,positionText,positionOrder,races.raceId,races.year,races.circuitId,races.name,drivers.forename,...,circuits.name,circuits.location,circuits.country,circuits.lat,circuits.lng,constructors.constructorRef,constructors.name,constructors.nationality,constructorId,drivers.fullname
4056,21326,863,813,R,23,863,2012,3,Bahrain Grand Prix,Pastor,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,williams,Williams,British,3,Pastor Maldonado
5373,21305,863,8,2,2,863,2012,3,Bahrain Grand Prix,Kimi,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,lotus_f1,Lotus F1,British,208,Kimi Räikkönen
6378,21306,863,154,3,3,863,2012,3,Bahrain Grand Prix,Romain,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,lotus_f1,Lotus F1,British,208,Romain Grosjean
8957,21325,863,811,22,22,863,2012,3,Bahrain Grand Prix,Bruno,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,williams,Williams,British,3,Bruno Senna
10092,21311,863,1,8,8,863,2012,3,Bahrain Grand Prix,Lewis,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,mclaren,McLaren,British,1,Lewis Hamilton
10247,21321,863,18,18,18,863,2012,3,Bahrain Grand Prix,Jenson,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,mclaren,McLaren,British,1,Jenson Button
13567,21318,863,817,15,15,863,2012,3,Bahrain Grand Prix,Daniel,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,toro_rosso,Toro Rosso,Italian,5,Daniel Ricciardo
13762,21317,863,818,14,14,863,2012,3,Bahrain Grand Prix,Jean-Éric,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,toro_rosso,Toro Rosso,Italian,5,Jean-Éric Vergne
16860,21310,863,4,7,7,863,2012,3,Bahrain Grand Prix,Fernando,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6,Fernando Alonso
17094,21312,863,13,9,9,863,2012,3,Bahrain Grand Prix,Felipe,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6,Felipe Massa


In [75]:
df_laps_circuit[df_laps_circuit['minute']=='1']

,raceId,driverId,lap,position,time,milliseconds,minute,seconds
27590,863,20,1,1,1:41.605,101605,1,41.605
27591,863,20,2,1,1:40.201,100201,1,40.201
27592,863,20,3,1,1:40.344,100344,1,40.344
27593,863,20,4,1,1:40.336,100336,1,40.336
27594,863,20,5,1,1:40.457,100457,1,40.457
...,...,...,...,...,...,...,...,...
515710,1074,817,53,16,1:37.671,97671,1,37.671
515711,1074,817,54,16,1:38.120,98120,1,38.120
515712,1074,817,55,15,1:38.639,98639,1,38.639
515713,1074,817,56,15,1:38.860,98860,1,38.860


In [65]:
df_laps_circuit['time'].split()

27590     1:41.605
27591     1:40.201
27592     1:40.344
27593     1:40.336
27594     1:40.457
            ...   
515710    1:37.671
515711    1:38.120
515712    1:38.639
515713    1:38.860
515714    1:40.084
Name: time, Length: 20489, dtype: object

In [ ]:
df_final[df_final['.name'] == circuit_name]

### Box Plot 

In [ ]:
df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('drivers.surname')['drivers.surname'].count().sort_values(ascending = False).index.tolist()

['Hamilton',
 'Vettel',
 'Alonso',
 'Massa',
 'Button',
 'Leclerc',
 'Pérez',
 'Rosberg',
 'Schumacher']

In [ ]:
def get_circuit_id(circuit_name):
    for index,circuit in enumerate(df_circuits): 
        if df_circuits['name'][index] == circuit_name:
            id_circuit = df_circuits['circuitId'][index]

    return id_circuit

In [ ]:
def get_driver_name(driver_id):
    for driver in df_drivers.index: 
        if driver == driver_id:
            driver_name = df_drivers['forename'][driver] + ' '+ df_drivers['surname'][driver]

    return driver_name

In [ ]:
def best_pilot_circuit(circuit): 
    # get the races in the circuit 
    race_list = df_races[df_races.circuitId == get_circuit_id(circuit_name)].index.tolist()
    # get the winning drivers from that circuit
    drivers_list = df_results[(df_results['raceId'].isin(race_list)) & (df_results['positionText'] == '1')].groupby('driverId')['driverId'].count().sort_values(ascending = False).index.tolist()
    # get the number of wins by each driver 
    winning_list = df_results[(df_results['raceId'].isin(race_list)) & (df_results['positionText'] == '1')].groupby('driverId')['driverId'].count().sort_values(ascending = False).tolist()
    
    # convert drivers id into names
    drivers_names = []
    for driver_id in drivers_list: 
        drivers_names.append(get_driver_name(driver_id))

    d = {'Drivers':drivers_names,'Wins':winning_list}
    df_wins = pd.DataFrame(d)

    fig = px.histogram(df, x="day", category_orders=dict(day=["Thur", "Fri", "Sat", "Sun"]))
    fig.show()
    

### Pie Chart 

In [100]:
df_final.columns

Index(['resultId', 'raceId', 'driverId', 'positionText', 'positionOrder',
       'races.raceId', 'races.year', 'races.circuitId', 'races.name',
       'drivers.forename', 'drivers.surname', 'drivers.nationality',
       'circuits.name', 'circuits.location', 'circuits.country',
       'circuits.lat', 'circuits.lng', 'constructors.constructorRef',
       'constructors.name', 'constructors.nationality', 'constructorId'],
      dtype='object')

In [101]:
const_wins = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).tolist()

In [102]:
const_names = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).index.tolist()

In [103]:
import plotly.express as px
# This dataframe has 244 lines, but 4 distinct values for `day`

fig = px.pie(values=const_wins, names=const_names,color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [104]:
df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).tolist()

[7, 6, 2, 2, 1, 1]

In [105]:
df = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['drivers.surname'].count()

In [106]:
contructors =df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby(['constructors.name', 'drivers.surname']).agg({'resultId': 'nunique'})

In [107]:
only_winners = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)]

In [108]:
only_winners

,resultId,raceId,driverId,positionText,positionOrder,races.raceId,races.year,races.circuitId,races.name,drivers.forename,...,drivers.nationality,circuits.name,circuits.location,circuits.country,circuits.lat,circuits.lng,constructors.constructorRef,constructors.name,constructors.nationality,constructorId
5519,24926,1046,815,1,1,1046,2020,3,Sakhir Grand Prix,Sergio,...,Mexican,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,racing_point,Racing Point,British,211
10907,7614,4,18,1,1,4,2009,3,Bahrain Grand Prix,Jenson,...,British,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,brawn,Brawn,British,23
16227,45,20,13,1,1,20,2008,3,Bahrain Grand Prix,Felipe,...,Brazilian,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16233,413,38,13,1,1,38,2007,3,Bahrain Grand Prix,Felipe,...,Brazilian,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16358,20323,337,4,1,1,337,2010,3,Bahrain Grand Prix,Fernando,...,Spanish,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16373,23419,971,20,1,1,971,2017,3,Bahrain Grand Prix,Sebastian,...,German,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16378,23802,990,20,1,1,990,2018,3,Bahrain Grand Prix,Sebastian,...,German,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16386,25406,1074,844,1,1,1074,2022,3,Bahrain Grand Prix,Charles,...,Monegasque,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16397,1555,92,30,1,1,92,2004,3,Bahrain Grand Prix,Michael,...,German,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
17826,743,53,4,1,1,53,2006,3,Bahrain Grand Prix,Fernando,...,Spanish,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,renault,Renault,French,4


In [109]:
contructors.reset_index(inplace = True)

In [110]:
contructors

,constructors.name,drivers.surname,resultId
0,Brawn,Button,1
1,Ferrari,Alonso,1
2,Ferrari,Leclerc,1
3,Ferrari,Massa,2
4,Ferrari,Schumacher,1
5,Ferrari,Vettel,2
6,Mercedes,Hamilton,5
7,Mercedes,Rosberg,1
8,Racing Point,Pérez,1
9,Red Bull,Vettel,2


In [76]:
import plotly.express as px
df = px.data.tips()
fig = px.sunburst(contructors, path=['constructors.name', 'drivers.surname'], values='total_bill')
fig.show()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().